In [31]:
import pandas as pd
import numpy as np


In [32]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
energy_efficiency = fetch_ucirepo(id=242) 
  
# data (as pandas dataframes) 
X = energy_efficiency.data.features 
y = energy_efficiency.data.targets 
  
# metadata 
print(energy_efficiency.metadata) 
  
# variable information 
print(energy_efficiency.variables) 

{'uci_id': 242, 'name': 'Energy Efficiency', 'repository_url': 'https://archive.ics.uci.edu/dataset/242/energy+efficiency', 'data_url': 'https://archive.ics.uci.edu/static/public/242/data.csv', 'abstract': 'This study looked into assessing the heating load and cooling load requirements of buildings (that is, energy efficiency) as a function of building parameters.', 'area': 'Computer Science', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate'], 'num_instances': 768, 'num_features': 8, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['Y1', 'Y2'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2012, 'last_updated': 'Mon Feb 26 2024', 'dataset_doi': '10.24432/C51307', 'creators': ['Athanasios Tsanas', 'Angeliki Xifara'], 'intro_paper': {'ID': 379, 'type': 'NATIVE', 'title': 'Accurate quantitative estimation of energy performance of residential buildings using statistical machine 

In [40]:
def train_valid_test_split(X, y):
    ''' Split the data into train, validation and test datasets using pandas'''
    train=int(X.shape[0]*0.7)
    valid=int(X.shape[0]*0.2)
    test=int(X.shape[0]*0.1)
    print("Data amounts to training data {}, validation data {} and testing data {}. ".format(train, valid, test))

    X_df=pd.DataFrame(X)
    X_df.insert(0, 'W0', 1)
    X_train=X_df.iloc[0:train,:].values
    X_valid=X_df.iloc[train:train+valid,:].values
    X_test=X_df.iloc[train+valid:,:].values
    y_df=pd.DataFrame(y)
    y_train=y_df.iloc[0:train,:].values
    y_valid=y_df.iloc[train:train+valid,:].values
    y_test=y_df.iloc[train+valid:,:].values
    
    return X_train, X_valid, X_test, y_train, y_valid, y_test

In [ ]:
def matrix_inverse(np_matrix):
    ''' Compute the matrix inverse using numpy'''
    np_matrix_inv=np.linalg.inv(np_matrix)
    return np_matrix_inv



In [ ]:
X_train, X_valid, X_test, y_train, y_valid, y_test=train_valid_test_split(X,y)
X_train

Data amounts to training data 537, validation data 153 and testing data 76. 


array([[1.000e+00, 9.800e-01, 5.145e+02, ..., 2.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 9.800e-01, 5.145e+02, ..., 3.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 9.800e-01, 5.145e+02, ..., 4.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [1.000e+00, 9.000e-01, 5.635e+02, ..., 4.000e+00, 4.000e-01,
        1.000e+00],
       [1.000e+00, 9.000e-01, 5.635e+02, ..., 5.000e+00, 4.000e-01,
        1.000e+00],
       [1.000e+00, 8.600e-01, 5.880e+02, ..., 2.000e+00, 4.000e-01,
        1.000e+00]])